In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import numpy as np
import joblib
import json

In [2]:
LGBMClassifier().get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [3]:
def dataset1_preprocessing(file_path,paramList,test_size):
    data = pd.read_csv(file_path)
    X = data[paramList]
    Y = data.condition
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=test_size)
    return x_train,x_test,y_train,y_test

In [4]:
def model_fit_forall(x,y):
    # RandonForest Classification
    param_search_RF = [{'n_estimators':[50,100,200,300],'max_depth':[None,4,8,12,16]},{'bootstrap':[False], 'n_estimators':[50,100,200,300],'max_depth':[None,4,8,12,16]}]
    model_RF = RandomForestClassifier()
    grid_search_RF = GridSearchCV(model_RF,param_search_RF,cv=3,n_jobs=-1)
    grid_search_RF.fit(x,y)
    # XGboost Classification
    param_search_XG={'learning_rate':[0.05,0.1,0.2,0.3],'n_estimators':[50,100,150]}
    model_XG = XGBClassifier()
    grid_search_XG = GridSearchCV(model_XG,param_search_XG,cv=3,n_jobs=-1)
    grid_search_XG.fit(x,y)
    # LGBMClassifier Classification
    param_search_LG={'learning_rate':[0.05,0.1,0.2,0.3],'n_estimators':[50,100,150]}
    model_XG = LGBMClassifier()
    grid_search_LG = GridSearchCV(model_XG,param_search_LG,cv=3,n_jobs=-1)
    grid_search_LG.fit(x,y)
    return grid_search_RF.best_estimator_,grid_search_XG.best_estimator_,grid_search_LG.best_estimator_

In [5]:
def find_model_forall(model_number,file_path,paramList,test_size,test):
    if(model_number==1):
        x_train,x_test,y_train,y_test = dataset1_preprocessing(file_path,paramList,test_size)
    if(model_number==2):
        x_train,x_test,y_train,y_test = dataset2_preprocessing(file_path,paramList,test_size)
    if(model_number==3):
        x_train,x_test,y_train,y_test = dataset3_preprocessing(file_path,paramList,test_size)
    result = []
    # Logistic Classification
    logisClassification = LogisticRegression().fit(x_train,y_train)
    result.append(logisClassification.score(x_test,y_test))
    # SVC Classification
    svcClassification = SVC(kernel='linear').fit(x_train,y_train)
    result.append(svcClassification.score(x_test,y_test))
    model_RF,model_XG,model_LG = model_fit_forall(x_train,y_train)
    # RandonFoest Classification
    model_RF.fit(x_train,y_train)
    result.append(model_RF.score(x_test,y_test))
    # XGBoost Classification
    model_XG.fit(x_train,y_train)
    result.append(model_XG.score(x_test,y_test))
    # LGBM Classification
    model_LG.fit(x_train,y_train)
    result.append(model_LG.score(x_test,y_test))
    
    #joblib save the model params
    joblib.dump(logisClassification,"logisClassification.pkl")
    joblib.dump(svcClassification,"svcClassification.pkl")
    joblib.dump(model_RF,"model_RF.pkl")
    joblib.dump(model_XG,"model_XG.pkl")
    joblib.dump(model_LG,"model_LG.pkl")
    
    max = np.argmax(result)
    testdata = np.array([test])
    
    if(max == 0):
        return logisClassification.predict(testdata),result
    if(max == 1):
        return svcClassification.predict(testdata),result
    if(max == 2):
        return model_RF.predict(testdata),result
    if(max == 3):
        return model_XG.predict(testdata),result
    if(max == 4):
        return model_LG.predict(testdata),result

In [ ]:
# input
paramList = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach','exang', 'oldpeak', 'slope', 'ca', 'thal']
number = [64,1,0,190,320,0,2,182,0,2.6,1,0,2]
# output
result,modelList = find_model_forall(1,'Data/dataset_1.csv',paramList,0.15,number)

/Users/sato/anaconda3/envs/machineLearning/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
def dataset2_preprocessing(file_path,paramList,test_size):
    df = pd.read_csv(file_path)
    df.loc[df['TenYearCHD'] == 1, 'education'] = df.loc[df['TenYearCHD'] == 1, 'education'].fillna(df.loc[df['TenYearCHD'] == 1, 'education'].mode()[0])
    df.loc[df['TenYearCHD'] == 0, 'education'] = df.loc[df['TenYearCHD'] == 0, 'education'].fillna(df.loc[df['TenYearCHD'] == 0, 'education'].mode()[0])
    df.loc[df['TenYearCHD'] == 1, 'cigsPerDay'] = df.loc[df['TenYearCHD'] == 1, 'cigsPerDay'].fillna(df.loc[df['TenYearCHD'] == 1, 'cigsPerDay'].mean())
    df.loc[df['TenYearCHD'] == 0, 'cigsPerDay'] = df.loc[df['TenYearCHD'] == 0, 'cigsPerDay'].fillna(df.loc[df['TenYearCHD'] == 0, 'cigsPerDay'].mean())
    df.loc[df['TenYearCHD'] == 1, 'BPMeds'] = df.loc[df['TenYearCHD'] == 1, 'BPMeds'].fillna(df.loc[df['TenYearCHD'] == 1, 'BPMeds'].mode()[0])
    df.loc[df['TenYearCHD'] == 0, 'BPMeds'] = df.loc[df['TenYearCHD'] == 0, 'BPMeds'].fillna(df.loc[df['TenYearCHD'] == 0, 'BPMeds'].mode()[0])
    df.loc[df['TenYearCHD'] == 1, 'totChol'] = df.loc[df['TenYearCHD'] == 1, 'totChol'].fillna(df.loc[df['TenYearCHD'] == 1, 'totChol'].mean())
    df.loc[df['TenYearCHD'] == 0, 'totChol'] = df.loc[df['TenYearCHD'] == 0, 'totChol'].fillna( df.loc[df['TenYearCHD'] == 0, 'totChol'].mean())
    df.loc[df['TenYearCHD'] == 1, 'BMI'] = df.loc[df['TenYearCHD'] == 1, 'BMI'].fillna(df.loc[df['TenYearCHD'] == 1, 'BMI'].mean())
    df.loc[df['TenYearCHD'] == 0, 'BMI'] = df.loc[df['TenYearCHD'] == 0, 'BMI'].fillna(df.loc[df['TenYearCHD'] == 0, 'BMI'].mean())
    df.loc[df['TenYearCHD'] == 1, 'heartRate'] = df.loc[df['TenYearCHD'] == 1, 'heartRate'].fillna(df.loc[df['TenYearCHD'] == 1, 'heartRate'].mean())
    df.loc[df['TenYearCHD'] == 0, 'heartRate'] = df.loc[df['TenYearCHD'] == 0, 'heartRate'].fillna(df.loc[df['TenYearCHD'] == 0, 'heartRate'].mean())
    df.loc[df['TenYearCHD'] == 1, 'glucose'] = df.loc[df['TenYearCHD'] == 1, 'glucose'].fillna(df.loc[df['TenYearCHD'] == 1, 'glucose'].mean())
    df.loc[df['TenYearCHD'] == 0, 'glucose'] = df.loc[df['TenYearCHD'] == 0, 'glucose'].fillna(df.loc[df['TenYearCHD'] == 0, 'glucose'].mean())
    X = df[paramList]
    Y = df.TenYearCHD
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=test_size)
    return x_train,x_test,y_train,y_test

In [ ]:
# input
paramList = ['male','age','education','currentSmoker','cigsPerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','diaBP','BMI','heartRate','glucose']
number = [0,43,2,0,0,0,0,0,0,247,131,88,27.64,72,61]
# output
result,modelList = find_model_forall(2,'Data/dataset_2.csv',paramList,0.15,number)

In [7]:
def dataset3_preprocessing(file_path,paramList,test_size):
    df = pd.read_csv(file_path,sep=';')
    df.age= round(df.age/365,0)
    X = df[paramList]
    Y = df.cardio
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=test_size)
    return x_train,x_test,y_train,y_test

In [ ]:
# input
paramList = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo','cholesterol', 'gluc', 'smoke', 'alco', 'active']
number = [39.81,2,181,95.0,130,90,1,1,1,1,1]
# output
result,modelList = find_model_forall(3,'Data/dataset_3.csv',paramList,0.15,number)

In [8]:
def dataset4_preprocessing(file_path,paramList,test_size):
    df4 = pd.read_csv(file_path)
    df4.Smoking = df4.Smoking.map({'No':0,'Yes':1})
    df4.HeartDisease = df4.HeartDisease.map({'No':0,'Yes':1})
    df4.AlcoholDrinking = df4.AlcoholDrinking.map({'No':0,'Yes':1})
    df4.Stroke = df4.Stroke.map({'No':0,'Yes':1})
    df4.DiffWalking = df4.DiffWalking.map({'No':0,'Yes':1})
    df4.Diabetic = df4.Diabetic.map({'No':0,'Yes':1})
    df4.loc[df4['HeartDisease'] == 1, 'Diabetic'] = df4.loc[df4['HeartDisease'] == 1, 'Diabetic'].fillna(df4.loc[df4['HeartDisease'] == 1, 'Diabetic'].mode()[0])
    df4.loc[df4['HeartDisease'] == 0, 'Diabetic'] = df4.loc[df4['HeartDisease'] == 0, 'Diabetic'].fillna(df4.loc[df4['HeartDisease'] == 0, 'Diabetic'].mode()[0])
    df4.PhysicalActivity = df4.PhysicalActivity.map({'No':0,'Yes':1})
    df4.Asthma = df4.Asthma.map({'No':0,'Yes':1})
    df4.KidneyDisease = df4.KidneyDisease.map({'No':0,'Yes':1})
    df4.SkinCancer = df4.SkinCancer.map({'No':0,'Yes':1})
    df4.Sex = df4.Sex.map({'Female':0,'Male':1})
    df4.AgeCategory = df4.AgeCategory.map({'55-59':57,'80 or older':82,'65-69':67,'75-79':77,'40-44':42,'70-74':72,'60-64':62,'50-54':52,'45-49':47,'18-24':21,'35-39':37,'30-34':32,'25-29':27})
    df4.Race = df4.Race.map({'White':1,'Black':2,'Asian':3,'American Indian/Alaskan Native':4,'Other':5,'Hispanic':6})
    df4.GenHealth = df4.GenHealth.map({'Very good':4, 'Fair':2, 'Good':3, 'Poor':1, 'Excellent':5})
    X = df4[paramList]
    Y = df4.HeartDisease
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=test_size)
    return x_train,x_test,y_train,y_test

In [ ]:
# input
paramList = ['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke','PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory','Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime','Asthma', 'KidneyDisease', 'SkinCancer']
number = [39.81,2,181,95.0,130,90,1,1,1,1,1]
# output
result,modelList = find_model_forall(4,'Data/dataset_4.csv',paramList,0.15,number)

In [9]:
def dataset5_preprocessing(file_path,paramList,test_size):
    df5 = pd.read_csv(file_path)
    df5.General_Health = df5.General_Health.map({'Very Good':4, 'Fair':2, 'Good':3, 'Poor':1, 'Excellent':5})
    df5.Checkup = df5.Checkup.map({'Within the past 2 years':2,'Within the past year':1,'5 or more years ago':4,'Within the past 5 years':3,'Never':0})
    df5.Exercise = df5.Exercise.map({'No':0,'Yes':1})
    df5.Heart_Disease = df5.Heart_Disease.map({'No':0,'Yes':1})
    df5.Skin_Cancer = df5.Skin_Cancer.map({'No':0,'Yes':1})
    df5.Other_Cancer = df5.Other_Cancer.map({'No':0,'Yes':1})
    df5.Depression = df5.Depression.map({'No':0,'Yes':1})
    df5.Arthritis = df5.Arthritis.map({'No':0,'Yes':1})
    df5.Diabetes = df5.Diabetes.map({'No':0,'Yes':1,'Yes, but female told only during pregnancy':1, 'No, pre-diabetes or borderline diabetes':0})
    df5.Smoking_History = df5.Smoking_History.map({'No':0,'Yes':1})
    df5.Sex = df5.Sex.map({'Female':0,'Male':1})
    df5.Age_Category = df5.Age_Category.map({'55-59':57,'80+':82,'65-69':67,'75-79':77,'40-44':42,'70-74':72,'60-64':62,'50-54':52,'45-49':47,'18-24':21,'35-39':37,'30-34':32,'25-29':27})
    X = df5[paramList]
    Y = df5.Heart_Disease
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=test_size)
    return x_train,x_test,y_train,y_test

In [ ]:
# input
paramList = ['General_Health', 'Checkup', 'Exercise', 'Skin_Cancer','Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex','Age_Category', 'Height_(cm)', 'Weight_(kg)', 'BMI', 'Smoking_History','Alcohol_Consumption', 'Fruit_Consumption','Green_Vegetables_Consumption', 'FriedPotato_Consumption']
number = [39.81,2,181,95.0,130,90,1,1,1,1,1]
# output
result,modelList = find_model_forall(5,'Data/dataset_5.csv',paramList,0.15,number)

In [10]:
def assignModel(jsondata):
    json_data = json.loads(jsondata)
    keys = json_data.keys()
    values = json_data.values()
    print(list(keys))

In [11]:
string = '{"name": "John", "age": 30, "city": "New York"}'
assignModel(string)

['name', 'age', 'city']


In [31]:
import tensorflow as tf
from matplotlib import pyplot as plt

# input
paramList = ['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke','PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory','Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime','Asthma', 'KidneyDisease', 'SkinCancer']
number = [39.81,2,181,95.0,130,90,1,1,1,1,1]

x_train,x_test,y_train,y_test = dataset4_preprocessing('Data/dataset_4.csv',paramList,0.15)

def one_hot(y_list):
    global _oneHot
    oneHot = []
    for y in y_list:
        if y==1:
            _oneHot = [1,0]
        if y==0:
            _oneHot = [0,1]
        oneHot.append(_oneHot)
    return np.array(oneHot)

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=100000)
dataset = dataset.batch(batch_size=100000)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


model_NN = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

for batch in dataset:
    features, labels = batch
    labels = one_hot(labels)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, verbose=1, restore_best_weights=True)
    model_NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model_NN.fit(features,labels,epochs=1000,callbacks=[early_stopping])

2023-08-04 11:56:18.225780: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [271825,17]
	 [[{{node Placeholder/_0}}]]
2023-08-04 11:56:18.226039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [271825]
	 [[{{node Placeholder/_1}}]]


Epoch 1/1000
3125/3125 [==============================] - 6s 2ms/step - loss: 0.2542 - accuracy: 0.9120
Epoch 2/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2343 - accuracy: 0.9150
Epoch 3/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2303 - accuracy: 0.9151
Epoch 4/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2291 - accuracy: 0.9158
Epoch 5/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2281 - accuracy: 0.9158
Epoch 6/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2275 - accuracy: 0.9157
Epoch 7/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2268 - accuracy: 0.9162
Epoch 8/1000
3125/3125 [==============================] - 4s 1ms/step - loss: 0.2261 - accuracy: 0.9162
Epoch 9/1000
1509/3125 [=============>................] - ETA: 2s - loss: 0.2267 - accuracy: 0.9161

KeyboardInterrupt: 

In [29]:
result2 = np.argmax(model_NN.predict([[1,61,1,0,0,0,0,1,0,201,164.5,93.5,27.73,78,95]]))
result2

1/1 [==============================] - 0s 80ms/step


1

In [30]:
a = model_NN.evaluate(x_test,one_hot(y_test))
a

20/20 [==============================] - 0s 968us/step - loss: 0.4152 - accuracy: 0.8333


[0.41524001955986023, 0.8333333134651184]